In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [13]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS revenue_zone,
    SUM(total_amount) AS revenur_monthly_total_amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    pickup_datetime >= '2020-01-01'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [25]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode="overwrite")

In [26]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [27]:
df_yellow.createOrReplaceTempView('yellow')

In [28]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS revenue_zone,
    SUM(total_amount) AS revenur_monthly_total_amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    pickup_datetime >= '2020-01-01'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [29]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode="overwrite")

In [31]:
pd.read_parquet('data/report/revenue/green')

,hour,revenue_zone,revenur_monthly_total_amount,number_records
0,2020-05-14 06:00:00,116,19.30,1
1,2020-01-15 14:00:00,217,56.80,1
2,2020-01-24 03:00:00,36,19.14,2
3,2020-02-12 11:00:00,257,23.41,1
4,2020-03-27 10:00:00,53,8.30,1
...,...,...,...,...
493778,2020-10-22 10:00:00,72,55.67,2
493779,2020-11-06 08:00:00,174,49.92,2
493780,2020-08-10 11:00:00,247,46.01,2
493781,2020-09-28 12:00:00,62,70.01,2
